In [ ]:
# Core data manipulation and analysis libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
import matplotlib.dates as mdates

# Date and time manipulation
from datetime import datetime, timedelta, date
import random

# Data quality and statistical analysis
from scipy import stats
import itertools

# Set visualization parameters for professional appearance
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

print("Environment setup complete. Libraries imported successfully.")
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)


In [ ]:
# Data generation parameters
NUM_RECORDS = 50000
MISSING_VALUE_RATE = 0.03  # 3% missing values
DATE_INCONSISTENCY_RATE = 0.01  # 1% date overlaps

# Define realistic data categories
regions = ['North America', 'Europe', 'Asia-Pacific', 'Latin America', 'Africa']
license_types = ['Exclusive', 'Non-Exclusive', 'Sync', 'Mechanical', 'Performance', 'Master Use']
licensing_channels = ['Digital Streaming', 'Broadcast TV', 'Radio', 'Film/TV Sync', 
                     'Gaming', 'Advertising', 'Live Performance', 'Mechanical']
compliance_statuses = ['Compliant', 'Non-Compliant', 'Under Review', 'Expired', 'Pending Renewal']

# Artist names (mix of realistic and fictional)
artist_names = [
    'The Midnight Echo', 'Sarah Chen', 'Digital Dreams', 'Marcus Rodriguez Band', 
    'Luna Park', 'Electric Horizon', 'The Velvet Underground Revival', 'Nina Aleksandrova',
    'Crimson Tide', 'Jazz Fusion Collective', 'The Indie Sessions', 'Ocean Drive',
    'Metropolitan Orchestra', 'Blue Note Ensemble', 'The Alternative', 'Neon Lights',
    'Classical Crossover', 'Urban Beat', 'The Folk Revival', 'Electronic Symphony',
    'Acoustic Sessions', 'The Rock Collective', 'Hip Hop Nation', 'Country Roads Band',
    'Pop Culture', 'The Blues Brothers Revival', 'Latin Heat', 'African Drums',
    'Asian Fusion', 'European Chamber Music', 'World Music Collective', 'The Experimentals',
    'Vintage Sound', 'Modern Classics', 'The Minimalists', 'Orchestral Pop',
    'Indie Folk', 'Electronic Jazz', 'Progressive Rock', 'Soul Revival'
]

# Song title patterns for realistic generation
song_prefixes = ['Lost in', 'Dancing with', 'Memories of', 'Love in', 'Journey to', 
                'Dreams of', 'Night in', 'Summer', 'Winter', 'Spring', 'Autumn',
                'City', 'Highway', 'River', 'Mountain', 'Ocean', 'Desert', 'Forest']
song_suffixes = ['the Night', 'Paradise', 'Tomorrow', 'Yesterday', 'the Stars', 'the Moon',
                'the Sun', 'the Rain', 'the Wind', 'the Fire', 'the Light', 'the Dark',
                'Blues', 'Dreams', 'Memories', 'Love', 'Time', 'Life', 'Hope', 'Freedom']

def generate_song_title():
    """Generate realistic song titles"""
    if random.random() < 0.3:
        return f"{random.choice(song_prefixes)} {random.choice(song_suffixes)}"
    elif random.random() < 0.6:
        return f"{random.choice(['The', 'My', 'Your', 'Our'])} {random.choice(song_suffixes)}"
    else:
        return f"{random.choice(song_prefixes)}"

print("Data generation parameters configured.")
print(f"Target records: {NUM_RECORDS:,}")
print(f"Missing value rate: {MISSING_VALUE_RATE:.1%}")
print(f"Date inconsistency rate: {DATE_INCONSISTENCY_RATE:.1%}")
print("Reference data categories loaded successfully.")


In [ ]:
# Generate the main dataset
print("Generating synthetic licensing dataset...")
start_time = datetime.now()

# Initialize data containers
licensing_data = []

# Define date ranges for realistic licensing periods
base_start_date = datetime(2020, 1, 1)
base_end_date = datetime(2024, 12, 31)

for i in range(NUM_RECORDS):
    # Generate unique identifiers
    license_id = f"LIC-{i+1:06d}"
    artist_id = random.randint(1001, 9999)
    song_id = random.randint(10001, 99999)
    
    # Generate artist and song information
    artist_name = random.choice(artist_names)
    song_title = generate_song_title()
    
    # Generate realistic date ranges
    start_date = base_start_date + timedelta(days=random.randint(0, 1460))  # Random start within 4 years
    
    # Most licenses are 1-5 years, with some shorter promotional licenses
    if random.random() < 0.7:
        duration_days = random.randint(365, 1825)  # 1-5 years
    else:
        duration_days = random.randint(30, 365)    # Short-term licenses
    
    end_date = start_date + timedelta(days=duration_days)
    
    # Introduce date inconsistencies (1% of records)
    if random.random() < DATE_INCONSISTENCY_RATE:
        # Make end date before start date
        end_date = start_date - timedelta(days=random.randint(1, 365))
    
    # Generate other attributes
    license_type = random.choice(license_types)
    licensing_channel = random.choice(licensing_channels)
    region = random.choice(regions)
    
    # Generate revenue with realistic distribution
    # Higher revenue for exclusive licenses and major channels
    base_revenue = np.random.lognormal(8, 1.5)  # Log-normal distribution for realistic revenue
    
    if license_type == 'Exclusive':
        base_revenue *= 2.5
    if licensing_channel in ['Digital Streaming', 'Broadcast TV']:
        base_revenue *= 1.8
    if region in ['North America', 'Europe']:
        base_revenue *= 1.4
    
    revenue_amount = round(base_revenue, 2)
    
    # Generate compliance status with realistic distribution
    compliance_weights = [0.65, 0.15, 0.08, 0.07, 0.05]  # Most are compliant
    compliance_status = np.random.choice(compliance_statuses, p=compliance_weights)
    
    # Create record
    record = {
        'license_id': license_id,
        'artist_id': artist_id,
        'artist_name': artist_name,
        'song_id': song_id,
        'song_title': song_title,
        'license_start_date': start_date.date(),
        'license_end_date': end_date.date(),
        'license_type': license_type,
        'licensing_channel': licensing_channel,
        'region': region,
        'revenue_amount': revenue_amount,
        'compliance_status': compliance_status
    }
    
    licensing_data.append(record)
    
    # Progress indicator
    if (i + 1) % 10000 == 0:
        print(f"Generated {i+1:,} records...")

# Create DataFrame
df_licensing = pd.DataFrame(licensing_data)

generation_time = datetime.now() - start_time
print(f"\nDataset generation completed in {generation_time.total_seconds():.2f} seconds")
print(f"Total records generated: {len(df_licensing):,}")
print(f"Dataset shape: {df_licensing.shape}")
print(f"Memory usage: {df_licensing.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


In [ ]:
# Introduce missing values (3% of records across specific columns)
print("\nIntroducing missing values to simulate real-world data quality issues...")

# Define columns that can have missing values (exclude primary keys)
missing_value_columns = ['artist_name', 'song_title', 'license_type', 'licensing_channel', 
                        'region', 'revenue_amount', 'compliance_status']

missing_count = 0
for column in missing_value_columns:
    # Calculate number of missing values for this column
    col_missing_count = int(len(df_licensing) * MISSING_VALUE_RATE / len(missing_value_columns))
    
    # Randomly select indices to make NaN
    missing_indices = np.random.choice(df_licensing.index, size=col_missing_count, replace=False)
    df_licensing.loc[missing_indices, column] = np.nan
    missing_count += col_missing_count

print(f"Introduced {missing_count:,} missing values across {len(missing_value_columns)} columns")
print(f"Missing value rate: {missing_count / (len(df_licensing) * len(missing_value_columns)):.2%}")

# Display basic dataset information
print("\n" + "="*60)
print("DATASET OVERVIEW")
print("="*60)
print(f"Dataset dimensions: {df_licensing.shape[0]:,} rows × {df_licensing.shape[1]} columns")
print(f"Date range: {df_licensing['license_start_date'].min()} to {df_licensing['license_end_date'].max()}")
print(f"Total revenue: ${df_licensing['revenue_amount'].sum():,.2f}")
print(f"Average revenue per license: ${df_licensing['revenue_amount'].mean():.2f}")

# Display sample records
print("\nSample Records:")
print(df_licensing.head().to_string())

# Data types
print("\nData Types:")
print(df_licensing.dtypes)


In [ ]:
# Data Quality Assessment
print("COMPREHENSIVE DATA QUALITY ASSESSMENT")
print("="*60)

# 1. Missing Value Analysis
print("\n1. MISSING VALUE ANALYSIS")
print("-" * 30)
missing_summary = df_licensing.isnull().sum()
missing_percentage = (missing_summary / len(df_licensing)) * 100

missing_df = pd.DataFrame({
    'Column': missing_summary.index,
    'Missing_Count': missing_summary.values,
    'Missing_Percentage': missing_percentage.values
}).round(2)

print(missing_df.to_string(index=False))

# 2. Date Consistency Analysis
print("\n2. DATE CONSISTENCY ANALYSIS")
print("-" * 30)

# Convert dates to datetime for analysis
df_licensing['license_start_date'] = pd.to_datetime(df_licensing['license_start_date'])
df_licensing['license_end_date'] = pd.to_datetime(df_licensing['license_end_date'])

# Check for date inconsistencies
date_issues = df_licensing['license_end_date'] < df_licensing['license_start_date']
inconsistent_dates = date_issues.sum()

print(f"Records with end date before start date: {inconsistent_dates:,} ({inconsistent_dates/len(df_licensing):.2%})")
print(f"Date range: {df_licensing['license_start_date'].min().date()} to {df_licensing['license_end_date'].max().date()}")

# Calculate license duration for valid dates
valid_dates = ~date_issues
df_licensing.loc[valid_dates, 'license_duration_days'] = (
    df_licensing.loc[valid_dates, 'license_end_date'] - 
    df_licensing.loc[valid_dates, 'license_start_date']
).dt.days

duration_stats = df_licensing['license_duration_days'].describe()
print(f"\nLicense Duration Statistics (days):")
print(f"Mean: {duration_stats['mean']:.0f}")
print(f"Median: {duration_stats['50%']:.0f}")
print(f"Min: {duration_stats['min']:.0f}")
print(f"Max: {duration_stats['max']:.0f}")

# 3. Revenue Analysis
print("\n3. REVENUE DISTRIBUTION ANALYSIS")
print("-" * 30)
revenue_stats = df_licensing['revenue_amount'].describe()
print(f"Revenue Statistics:")
for stat, value in revenue_stats.items():
    if pd.notna(value):
        print(f"{stat.capitalize()}: ${value:,.2f}")

# Identify outliers (using IQR method)
Q1 = df_licensing['revenue_amount'].quantile(0.25)
Q3 = df_licensing['revenue_amount'].quantile(0.75)
IQR = Q3 - Q1
outlier_threshold_high = Q3 + 1.5 * IQR
outlier_threshold_low = Q1 - 1.5 * IQR

outliers = df_licensing[
    (df_licensing['revenue_amount'] > outlier_threshold_high) | 
    (df_licensing['revenue_amount'] < outlier_threshold_low)
]

print(f"\nRevenue outliers detected: {len(outliers):,} ({len(outliers)/len(df_licensing):.2%})")
print(f"Outlier threshold: ${outlier_threshold_low:.2f} - ${outlier_threshold_high:.2f}")

# 4. Categorical Data Distribution
print("\n4. CATEGORICAL DATA DISTRIBUTION")
print("-" * 30)

categorical_columns = ['license_type', 'licensing_channel', 'region', 'compliance_status']

for col in categorical_columns:
    print(f"\n{col.upper()} DISTRIBUTION:")
    distribution = df_licensing[col].value_counts(dropna=False)
    percentage = df_licensing[col].value_counts(normalize=True, dropna=False) * 100
    
    for category, count in distribution.items():
        pct = percentage[category]
        print(f"  {str(category):<20}: {count:>8,} ({pct:>5.1f}%)")

print("\n" + "="*60)


In [ ]:
# Create a copy of the original dataset for cleaning
df_clean = df_licensing.copy()
cleaning_log = []

print("STARTING DATA CLEANING PROCESS")
print("="*60)

# 1. Fix Date Inconsistencies
print("\n1. CORRECTING DATE INCONSISTENCIES")
print("-" * 40)

date_issues = df_clean['license_end_date'] < df_clean['license_start_date']
inconsistent_count = date_issues.sum()

if inconsistent_count > 0:
    print(f"Fixing {inconsistent_count:,} records with invalid date ranges...")
    
    # Fix by adding a realistic duration to start date
    for idx in df_clean[date_issues].index:
        start_date = df_clean.loc[idx, 'license_start_date']
        # Add 1-3 years as a reasonable license duration
        duration_days = random.randint(365, 1095)
        df_clean.loc[idx, 'license_end_date'] = start_date + timedelta(days=duration_days)
    
    cleaning_log.append(f"Fixed {inconsistent_count} date inconsistencies")
    print(f"Date inconsistencies resolved")

# Recalculate license duration
df_clean['license_duration_days'] = (
    df_clean['license_end_date'] - df_clean['license_start_date']
).dt.days

# 2. Handle Missing Values
print("\n2. HANDLING MISSING VALUES")
print("-" * 40)

# Missing value counts before cleaning
missing_before = df_clean.isnull().sum()

# 2a. Artist and Song Names - Forward fill based on IDs
artist_missing = df_clean['artist_name'].isnull().sum()
song_missing = df_clean['song_title'].isnull().sum()

if artist_missing > 0:
    # Group by artist_id and forward fill
    df_clean['artist_name'] = df_clean.groupby('artist_id')['artist_name'].transform(
        lambda x: x.fillna(method='ffill').fillna(method='bfill')
    )
    # If still missing, use generic name
    df_clean['artist_name'] = df_clean['artist_name'].fillna('Unknown Artist')
    print(f"Filled {artist_missing:,} missing artist names")

if song_missing > 0:
    df_clean['song_title'] = df_clean.groupby('song_id')['song_title'].transform(
        lambda x: x.fillna(method='ffill').fillna(method='bfill')
    )
    df_clean['song_title'] = df_clean['song_title'].fillna('Unknown Title')
    print(f"Filled {song_missing:,} missing song titles")

# 2b. Categorical fields - Mode imputation
categorical_fields = ['license_type', 'licensing_channel', 'region', 'compliance_status']

for field in categorical_fields:
    missing_count = df_clean[field].isnull().sum()
    if missing_count > 0:
        mode_value = df_clean[field].mode()[0]
        df_clean[field] = df_clean[field].fillna(mode_value)
        print(f"Filled {missing_count:,} missing {field} values with mode: '{mode_value}'")

# 2c. Revenue amount - Median imputation by license type and region
revenue_missing = df_clean['revenue_amount'].isnull().sum()
if revenue_missing > 0:
    # Calculate median by license_type and region
    median_revenue = df_clean.groupby(['license_type', 'region'])['revenue_amount'].median()
    
    for idx in df_clean[df_clean['revenue_amount'].isnull()].index:
        license_type = df_clean.loc[idx, 'license_type']
        region = df_clean.loc[idx, 'region']
        
        if (license_type, region) in median_revenue.index:
            df_clean.loc[idx, 'revenue_amount'] = median_revenue[(license_type, region)]
        else:
            # Use overall median if group median not available
            df_clean.loc[idx, 'revenue_amount'] = df_clean['revenue_amount'].median()
    
    print(f"Filled {revenue_missing:,} missing revenue amounts using median imputation")

# 3. Data Standardization
print("\n3. DATA STANDARDIZATION")
print("-" * 40)

# 3a. Standardize artist and song names
original_artist_count = len(df_clean['artist_name'].unique())
original_song_count = len(df_clean['song_title'].unique())

# Clean and standardize text
df_clean['artist_name'] = df_clean['artist_name'].str.strip().str.title()
df_clean['song_title'] = df_clean['song_title'].str.strip().str.title()

new_artist_count = len(df_clean['artist_name'].unique())
new_song_count = len(df_clean['song_title'].unique())

print(f"Standardized artist names: {original_artist_count} → {new_artist_count} unique values")
print(f"Standardized song titles: {original_song_count} → {new_song_count} unique values")

# 3b. Ensure categorical consistency
df_clean['license_type'] = df_clean['license_type'].str.strip()
df_clean['licensing_channel'] = df_clean['licensing_channel'].str.strip()
df_clean['region'] = df_clean['region'].str.strip()
df_clean['compliance_status'] = df_clean['compliance_status'].str.strip()

# Summary of cleaning results
missing_after = df_clean.isnull().sum()
print(f"\n4. CLEANING SUMMARY")
print("-" * 40)
print(f"Records processed: {len(df_clean):,}")
print(f"Total missing values before: {missing_before.sum():,}")
print(f"Total missing values after: {missing_after.sum():,}")
print(f"Missing values resolved: {missing_before.sum() - missing_after.sum():,}")
print(f"Data completeness: {((len(df_clean) * len(df_clean.columns) - missing_after.sum()) / (len(df_clean) * len(df_clean.columns))) * 100:.2f}%")

print("\nDATA CLEANING COMPLETED SUCCESSFULLY")
print("="*60)


In [ ]:
# Calculate Key Performance Indicators
print("CALCULATING KEY PERFORMANCE INDICATORS")
print("="*60)

# 1. Total Revenue by Artist
print("\n1. REVENUE BY ARTIST ANALYSIS")
print("-" * 40)

artist_revenue = df_clean.groupby('artist_name').agg({
    'revenue_amount': ['sum', 'count', 'mean'],
    'license_duration_days': 'mean'
}).round(2)

artist_revenue.columns = ['Total_Revenue', 'License_Count', 'Avg_Revenue_Per_License', 'Avg_Duration_Days']
artist_revenue = artist_revenue.sort_values('Total_Revenue', ascending=False)

# Top 10 artists by revenue
top_artists = artist_revenue.head(10)
print("TOP 10 REVENUE-GENERATING ARTISTS:")
print(f"{'Rank':<4} {'Artist Name':<25} {'Total Revenue':<15} {'Licenses':<10} {'Avg Revenue':<12}")
print("-" * 75)

for i, (artist, data) in enumerate(top_artists.iterrows(), 1):
    print(f"{i:<4} {artist:<25} ${data['Total_Revenue']:>12,.0f} {data['License_Count']:>8,.0f} ${data['Avg_Revenue_Per_License']:>10,.0f}")

total_revenue = df_clean['revenue_amount'].sum()
top_10_revenue = top_artists['Total_Revenue'].sum()
print(f"\nTop 10 artists represent ${top_10_revenue:,.0f} ({top_10_revenue/total_revenue:.1%}) of total revenue")

# 2. Revenue by Region
print("\n2. REGIONAL REVENUE ANALYSIS")
print("-" * 40)

regional_revenue = df_clean.groupby('region').agg({
    'revenue_amount': ['sum', 'count', 'mean'],
    'license_duration_days': 'mean'
}).round(2)

regional_revenue.columns = ['Total_Revenue', 'License_Count', 'Avg_Revenue_Per_License', 'Avg_Duration_Days']
regional_revenue = regional_revenue.sort_values('Total_Revenue', ascending=False)

print("REVENUE BY REGION:")
print(f"{'Region':<20} {'Total Revenue':<15} {'Licenses':<10} {'Market Share':<12} {'Avg Revenue':<12}")
print("-" * 80)

for region, data in regional_revenue.iterrows():
    market_share = (data['Total_Revenue'] / total_revenue) * 100
    print(f"{region:<20} ${data['Total_Revenue']:>12,.0f} {data['License_Count']:>8,.0f} {market_share:>10.1f}% ${data['Avg_Revenue_Per_License']:>10,.0f}")

# 3. Average License Duration Analysis
print("\n3. LICENSE DURATION ANALYSIS")
print("-" * 40)

duration_stats = df_clean['license_duration_days'].describe()
print("OVERALL LICENSE DURATION STATISTICS:")
print(f"Mean Duration: {duration_stats['mean']:.0f} days ({duration_stats['mean']/365:.1f} years)")
print(f"Median Duration: {duration_stats['50%']:.0f} days ({duration_stats['50%']/365:.1f} years)")
print(f"Standard Deviation: {duration_stats['std']:.0f} days")
print(f"Range: {duration_stats['min']:.0f} - {duration_stats['max']:.0f} days")

# Duration by license type
duration_by_type = df_clean.groupby('license_type')['license_duration_days'].agg(['mean', 'median', 'count']).round(0)
print(f"\nDURATION BY LICENSE TYPE:")
print(f"{'License Type':<20} {'Mean (days)':<12} {'Median (days)':<15} {'Count':<8}")
print("-" * 60)
for license_type, data in duration_by_type.iterrows():
    print(f"{license_type:<20} {data['mean']:>10.0f} {data['median']:>13.0f} {data['count']:>6.0f}")

# 4. License Compliance Rates
print("\n4. COMPLIANCE RATE ANALYSIS")
print("-" * 40)

compliance_analysis = df_clean['compliance_status'].value_counts()
compliance_percentages = df_clean['compliance_status'].value_counts(normalize=True) * 100

print("OVERALL COMPLIANCE STATUS:")
print(f"{'Status':<20} {'Count':<10} {'Percentage':<12}")
print("-" * 45)
for status, count in compliance_analysis.items():
    percentage = compliance_percentages[status]
    print(f"{status:<20} {count:>8,} {percentage:>10.1f}%")

# Compliance by region
compliance_by_region = pd.crosstab(df_clean['region'], df_clean['compliance_status'], normalize='index') * 100
print(f"\nCOMPLIANCE RATES BY REGION (%):")
print(compliance_by_region.round(1).to_string())

# 5. Revenue by Licensing Channel
print("\n5. LICENSING CHANNEL PERFORMANCE")
print("-" * 40)

channel_revenue = df_clean.groupby('licensing_channel').agg({
    'revenue_amount': ['sum', 'count', 'mean'],
    'license_duration_days': 'mean'
}).round(2)

channel_revenue.columns = ['Total_Revenue', 'License_Count', 'Avg_Revenue_Per_License', 'Avg_Duration_Days']
channel_revenue = channel_revenue.sort_values('Total_Revenue', ascending=False)

print("REVENUE BY LICENSING CHANNEL:")
print(f"{'Channel':<20} {'Total Revenue':<15} {'Licenses':<10} {'Market Share':<12} {'Avg Revenue':<12}")
print("-" * 80)

for channel, data in channel_revenue.iterrows():
    market_share = (data['Total_Revenue'] / total_revenue) * 100
    print(f"{channel:<20} ${data['Total_Revenue']:>12,.0f} {data['License_Count']:>8,.0f} {market_share:>10.1f}% ${data['Avg_Revenue_Per_License']:>10,.0f}")

# 6. Executive Summary KPIs
print("\n6. EXECUTIVE SUMMARY METRICS")
print("-" * 40)

total_licenses = len(df_clean)
unique_artists = df_clean['artist_name'].nunique()
unique_songs = df_clean['song_title'].nunique()
avg_revenue_per_license = df_clean['revenue_amount'].mean()
total_revenue_millions = total_revenue / 1_000_000
compliant_rate = (compliance_analysis.get('Compliant', 0) / total_licenses) * 100

print(f"Total Portfolio Value: ${total_revenue_millions:.1f}M")
print(f"Active Licenses: {total_licenses:,}")
print(f"Unique Artists: {unique_artists:,}")
print(f"Unique Songs: {unique_songs:,}")
print(f"Average Revenue per License: ${avg_revenue_per_license:,.0f}")
print(f"Overall Compliance Rate: {compliant_rate:.1f}%")
print(f"Average License Duration: {duration_stats['mean']/365:.1f} years")

print("\nKPI ANALYSIS COMPLETED")
print("="*60)


In [ ]:
# Executive Dashboard Visualizations
print("GENERATING EXECUTIVE DASHBOARD VISUALIZATIONS")
print("="*60)

# Set up professional color palettes
colors_primary = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
colors_secondary = ['#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
colors_professional = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#5A189A']

# Create a comprehensive dashboard
fig = plt.figure(figsize=(20, 16))
gs = GridSpec(3, 3, figure=fig, hspace=0.3, wspace=0.3)

# 1. Top 10 Revenue-Generating Artists (Horizontal Bar Chart)
ax1 = fig.add_subplot(gs[0, :2])
top_10_artists = artist_revenue.head(10)
bars = ax1.barh(range(len(top_10_artists)), top_10_artists['Total_Revenue'], 
                color=colors_professional[0], alpha=0.8)
ax1.set_yticks(range(len(top_10_artists)))
ax1.set_yticklabels([name[:20] + '...' if len(name) > 20 else name for name in top_10_artists.index])
ax1.set_xlabel('Total Revenue (USD)', fontsize=12, fontweight='bold')
ax1.set_title('Top 10 Revenue-Generating Artists', fontsize=14, fontweight='bold', pad=20)
ax1.grid(axis='x', alpha=0.3)

# Add value labels on bars
for i, bar in enumerate(bars):
    width = bar.get_width()
    ax1.text(width + width*0.01, bar.get_y() + bar.get_height()/2,
             f'${width:,.0f}', ha='left', va='center', fontweight='bold', fontsize=10)

# 2. Regional Revenue Distribution (Pie Chart)
ax2 = fig.add_subplot(gs[0, 2])
regional_data = regional_revenue['Total_Revenue']
wedges, texts, autotexts = ax2.pie(regional_data.values, labels=regional_data.index, 
                                   autopct='%1.1f%%', startangle=90, colors=colors_professional)
ax2.set_title('Revenue Distribution by Region', fontsize=14, fontweight='bold', pad=20)

# Enhance pie chart appearance
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')
    autotext.set_fontsize(10)

# 3. License Type Distribution (Donut Chart)
ax3 = fig.add_subplot(gs[1, 0])
license_type_counts = df_clean['license_type'].value_counts()
wedges, texts, autotexts = ax3.pie(license_type_counts.values, labels=license_type_counts.index,
                                   autopct='%1.1f%%', startangle=90, colors=colors_secondary,
                                   pctdistance=0.85)

# Create donut by adding a white circle in the center
centre_circle = plt.Circle((0, 0), 0.50, fc='white')
ax3.add_artist(centre_circle)
ax3.set_title('License Type Distribution', fontsize=14, fontweight='bold', pad=20)

for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')
    autotext.set_fontsize(9)

# 4. Compliance Status Overview (Stacked Bar Chart)
ax4 = fig.add_subplot(gs[1, 1])
compliance_by_region_counts = pd.crosstab(df_clean['region'], df_clean['compliance_status'])
compliance_by_region_counts.plot(kind='bar', stacked=True, ax=ax4, 
                                 color=colors_professional, alpha=0.8)
ax4.set_title('Compliance Status by Region', fontsize=14, fontweight='bold', pad=20)
ax4.set_xlabel('Region', fontsize=12, fontweight='bold')
ax4.set_ylabel('Number of Licenses', fontsize=12, fontweight='bold')
ax4.legend(title='Compliance Status', bbox_to_anchor=(1.05, 1), loc='upper left')
ax4.tick_params(axis='x', rotation=45)

# 5. Licensing Channel Performance (Horizontal Bar Chart)
ax5 = fig.add_subplot(gs[1, 2])
channel_data = channel_revenue.sort_values('Total_Revenue', ascending=True)
bars = ax5.barh(range(len(channel_data)), channel_data['Total_Revenue'], 
                color=colors_professional[1], alpha=0.8)
ax5.set_yticks(range(len(channel_data)))
ax5.set_yticklabels([name[:15] + '...' if len(name) > 15 else name for name in channel_data.index])
ax5.set_xlabel('Total Revenue (USD)', fontsize=12, fontweight='bold')
ax5.set_title('Revenue by Licensing Channel', fontsize=14, fontweight='bold', pad=20)
ax5.grid(axis='x', alpha=0.3)

# Add value labels on bars
for i, bar in enumerate(bars):
    width = bar.get_width()
    ax5.text(width + width*0.01, bar.get_y() + bar.get_height()/2,
             f'${width:,.0f}', ha='left', va='center', fontweight='bold', fontsize=9)

# 6. Revenue Distribution Histogram
ax6 = fig.add_subplot(gs[2, 0])
ax6.hist(df_clean['revenue_amount'], bins=50, color=colors_professional[2], alpha=0.7, edgecolor='white')
ax6.set_xlabel('Revenue Amount (USD)', fontsize=12, fontweight='bold')
ax6.set_ylabel('Frequency', fontsize=12, fontweight='bold')
ax6.set_title('Revenue Amount Distribution', fontsize=14, fontweight='bold', pad=20)
ax6.grid(alpha=0.3)

# Add median line
median_revenue = df_clean['revenue_amount'].median()
ax6.axvline(median_revenue, color='red', linestyle='--', linewidth=2, 
            label=f'Median: ${median_revenue:,.0f}')
ax6.legend()

# 7. License Duration Analysis
ax7 = fig.add_subplot(gs[2, 1])
duration_by_type_chart = df_clean.groupby('license_type')['license_duration_days'].mean().sort_values()
bars = ax7.bar(range(len(duration_by_type_chart)), duration_by_type_chart.values/365, 
               color=colors_professional[3], alpha=0.8)
ax7.set_xticks(range(len(duration_by_type_chart)))
ax7.set_xticklabels([name[:10] + '...' if len(name) > 10 else name for name in duration_by_type_chart.index], 
                    rotation=45, ha='right')
ax7.set_ylabel('Average Duration (Years)', fontsize=12, fontweight='bold')
ax7.set_title('Average License Duration by Type', fontsize=14, fontweight='bold', pad=20)
ax7.grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax7.text(bar.get_x() + bar.get_width()/2, height + height*0.01,
             f'{height:.1f}y', ha='center', va='bottom', fontweight='bold', fontsize=10)

# 8. Compliance Rate Gauge Chart (Simplified)
ax8 = fig.add_subplot(gs[2, 2])
compliance_rate = (compliance_analysis.get('Compliant', 0) / len(df_clean)) * 100
non_compliance_rate = 100 - compliance_rate

# Create a pie chart that looks like a gauge
sizes = [compliance_rate, non_compliance_rate]
colors_gauge = ['#2ca02c', '#d62728']
wedges, texts = ax8.pie(sizes, startangle=90, colors=colors_gauge, counterclock=False)

# Add center circle to make it look like a gauge
centre_circle = plt.Circle((0, 0), 0.50, fc='white')
ax8.add_artist(centre_circle)

# Add compliance rate text in center
ax8.text(0, 0, f'{compliance_rate:.1f}%\nCompliant', ha='center', va='center', 
         fontsize=16, fontweight='bold', color=colors_gauge[0])

ax8.set_title('Overall Compliance Rate', fontsize=14, fontweight='bold', pad=20)

# Main title for the entire dashboard
fig.suptitle('Music Licensing Performance Dashboard - Executive Summary', 
             fontsize=20, fontweight='bold', y=0.98)

plt.tight_layout()
plt.show()

print("Executive dashboard visualizations generated successfully")
print("="*60)


In [ ]:
# Data Export Process
print("INITIATING DATA EXPORT PROCESS")
print("="*60)

# 1. Export cleaned main dataset
print("\n1. EXPORTING CLEANED DATASET")
print("-" * 40)

# Prepare final dataset for export
df_export = df_clean.copy()

# Format dates for export
df_export['license_start_date'] = df_export['license_start_date'].dt.strftime('%Y-%m-%d')
df_export['license_end_date'] = df_export['license_end_date'].dt.strftime('%Y-%m-%d')

# Round numeric columns
df_export['revenue_amount'] = df_export['revenue_amount'].round(2)
df_export['license_duration_days'] = df_export['license_duration_days'].round(0)

# Export main dataset
main_export_filename = 'music_licensing_cleaned_dataset.csv'
df_export.to_csv(main_export_filename, index=False)
print(f"Main dataset exported: {main_export_filename}")
print(f"  Records: {len(df_export):,}")
print(f"  Columns: {len(df_export.columns)}")
print(f"  File size: {round(len(df_export.to_csv(index=False).encode('utf-8')) / 1024**2, 2)} MB")

# 2. Export KPI summary tables
print("\n2. EXPORTING KPI SUMMARY TABLES")
print("-" * 40)

# Artist revenue summary
artist_summary_filename = 'artist_revenue_summary.csv'
artist_revenue.round(2).to_csv(artist_summary_filename)
print(f"Artist revenue summary exported: {artist_summary_filename}")

# Regional performance summary
regional_summary_filename = 'regional_performance_summary.csv'
regional_revenue.round(2).to_csv(regional_summary_filename)
print(f"Regional performance summary exported: {regional_summary_filename}")

# Channel performance summary
channel_summary_filename = 'channel_performance_summary.csv'
channel_revenue.round(2).to_csv(channel_summary_filename)
print(f"Channel performance summary exported: {channel_summary_filename}")

# Compliance summary
compliance_summary = pd.DataFrame({
    'Compliance_Status': compliance_analysis.index,
    'Count': compliance_analysis.values,
    'Percentage': compliance_percentages.values
}).round(2)
compliance_summary_filename = 'compliance_summary.csv'
compliance_summary.to_csv(compliance_summary_filename, index=False)
print(f"Compliance summary exported: {compliance_summary_filename}")

# 3. Create data dictionary
print("\n3. CREATING DATA DICTIONARY")
print("-" * 40)

data_dictionary = pd.DataFrame({
    'Column_Name': [
        'license_id', 'artist_id', 'artist_name', 'song_id', 'song_title',
        'license_start_date', 'license_end_date', 'license_type', 
        'licensing_channel', 'region', 'revenue_amount', 'compliance_status',
        'license_duration_days'
    ],
    'Data_Type': [
        'String', 'Integer', 'String', 'Integer', 'String',
        'Date', 'Date', 'String', 'String', 'String', 'Float', 'String', 'Integer'
    ],
    'Description': [
        'Unique identifier for each licensing agreement',
        'Unique identifier for artist',
        'Name of the artist or musical act',
        'Unique identifier for song',
        'Title of the licensed song',
        'Start date of licensing agreement',
        'End date of licensing agreement',
        'Type of licensing arrangement (Exclusive, Non-Exclusive, etc.)',
        'Distribution channel for the license',
        'Geographic region of the license',
        'Revenue amount generated in USD',
        'Current compliance status of the license',
        'Duration of license in days (calculated field)'
    ],
    'Business_Rules': [
        'Format: LIC-XXXXXX (6-digit sequence)',
        'Numeric identifier, links to artist master data',
        'Standardized format (Title Case)',
        'Numeric identifier, links to song master data', 
        'Standardized format (Title Case)',
        'YYYY-MM-DD format, must be valid date',
        'YYYY-MM-DD format, must be >= start_date',
        'Controlled vocabulary: Exclusive, Non-Exclusive, Sync, Mechanical, Performance, Master Use',
        'Controlled vocabulary: Digital Streaming, Broadcast TV, Radio, Film/TV Sync, Gaming, Advertising, Live Performance, Mechanical',
        'Controlled vocabulary: North America, Europe, Asia-Pacific, Latin America, Africa',
        'Positive numeric value, 2 decimal places',
        'Controlled vocabulary: Compliant, Non-Compliant, Under Review, Expired, Pending Renewal',
        'Calculated as end_date - start_date, minimum 0 days'
    ]
})

dictionary_filename = 'music_licensing_data_dictionary.csv'
data_dictionary.to_csv(dictionary_filename, index=False)
print(f"Data dictionary exported: {dictionary_filename}")

# 4. Export analysis summary
print("\n4. CREATING ANALYSIS SUMMARY REPORT")
print("-" * 40)

analysis_summary = pd.DataFrame({
    'Metric': [
        'Total Licenses', 'Unique Artists', 'Unique Songs', 'Total Revenue (USD)',
        'Average Revenue per License (USD)', 'Average License Duration (Years)',
        'Compliance Rate (%)', 'Data Completeness (%)', 'Date Inconsistencies Fixed',
        'Missing Values Resolved', 'Top Revenue Region', 'Top Revenue Channel'
    ],
    'Value': [
        f"{len(df_clean):,}",
        f"{df_clean['artist_name'].nunique():,}",
        f"{df_clean['song_title'].nunique():,}",
        f"${df_clean['revenue_amount'].sum():,.2f}",
        f"${df_clean['revenue_amount'].mean():,.2f}",
        f"{df_clean['license_duration_days'].mean()/365:.1f}",
        f"{(compliance_analysis.get('Compliant', 0) / len(df_clean)) * 100:.1f}%",
        f"{((len(df_clean) * len(df_clean.columns) - df_clean.isnull().sum().sum()) / (len(df_clean) * len(df_clean.columns))) * 100:.1f}%",
        f"{inconsistent_count:,}" if inconsistent_count > 0 else "0",
        f"{missing_before.sum() - df_clean.isnull().sum().sum():,}",
        regional_revenue.index[0],
        channel_revenue.index[0]
    ]
})

summary_filename = 'licensing_analysis_summary.csv'
analysis_summary.to_csv(summary_filename, index=False)
print(f"Analysis summary exported: {summary_filename}")

# 5. Display export summary
print("\n5. EXPORT SUMMARY")
print("-" * 40)
print("Files Created:")
print(f"  • {main_export_filename} - Main cleaned dataset")
print(f"  • {artist_summary_filename} - Artist performance metrics")
print(f"  • {regional_summary_filename} - Regional analysis")
print(f"  • {channel_summary_filename} - Channel performance data")
print(f"  • {compliance_summary_filename} - Compliance status breakdown")
print(f"  • {dictionary_filename} - Data field definitions")
print(f"  • {summary_filename} - Executive summary metrics")

print(f"\nTotal files exported: 7")
print(f"Primary dataset ready for dashboard integration")

print("\nDATA EXPORT COMPLETED SUCCESSFULLY")
print("="*60)
